In [48]:
import pandas as pd, altair as alt

# Add '..' to the path to go up one level
import sys
sys.path.append('..')

import altair_wrapper.eco_styles as eco_styles
styles = eco_styles.EcoStyles()

styles.register_and_enable_theme(dark_mode=True)

---

#### UK Labour Market

**Target**: Build a selection of charts using data directly from ECO API.

We'll select only data sources that are updated monthly in the ONS Labour Force Survey.

Charts:
- Weekly wage: `wage`
- Redundancy rate: `redn`
- Unemployment rate: `unem`
- Inactivity: `inac`
- Employed: `empd`
- Employment rate: `empl`
- Hours worked: `hours`

In [199]:
lfs = {
    'unem': {
        'eco-code': 'unem',
        'label': 'Unemployment rate',
        'title': 'Unemployment',
        'subtitle': 'Rate, ages 16 and over, SA | Source: ONS via ECO API',
        'units': '%',
        'units-label': '.1%',
        'label-expression': 'format(datum.value, ".0%")',
        'text-expression': 'format(datum.value, ".1%")',
        'domain': [0, 0.06]
    },
    'vacn': {
        'eco-code': 'vacn',
        'label': 'Job Vacancies',
        'title': 'Job Vacancies',
        'subtitle': "Total vacancies, 000's | Source: ONS via ECO API",
        'units': 'd',
        'units-label': '.1',
        'label-expression': 'format(datum.value, ".0f")',
        'text-expression': 'format(datum.value, ".0f") + "k"'
    },
    'redn': {
        'eco-code': 'redn',
        'label': 'Redundancy rate',
        'title': 'Redundancies',
        'subtitle': 'Per 1000 employees, SA | Source: ONS via ECO API',
        'units': 'd',
        'units-label': '.1',
        'label-expression': 'format(datum.value, ".0f")',
        'text-expression': 'format(datum.value, ".1f")'
    },
    'empl': {
        'eco-code': 'empl',
        'label': 'Employment rate',
        'title': 'Employment',
        'subtitle': 'Rate, ages 16 and over, SA | Source: ONS via ECO API',
        'units': '%',
        'units-label': '.1%',
        'label-expression': 'format(datum.value, ".0%")',
        'text-expression': 'format(datum.value, ".1%")',
        'domain': [0.55, 0.66]
    },
    'empd': {
        'eco-code': 'empd',
        'label': 'Employed',
        'title': 'Workers',
        'subtitle': 'Persons employed, 16+, seasonally adjusted | Source: ONS via ECO API',
        'units': 'd',
        'units-label': '.1s',
        'label-expression': 'format(datum.value / 1000,  ".0f") + "m"',
        'text-expression': 'format(datum.value / 1000,  ".1f") + "m"',
        'domain': [26000, 34000]
    },
    'inac': {
        'eco-code': 'inac',
        'label': 'inactivity rate',
        'title': 'Economic Inactivity',
        'subtitle': '% of workforce, aged 16-64, SA | Source: ONS via ECO API',
        'units': '%',
        'units-label': '.1%',
        'label-expression': 'format(datum.value, ".0%")',
        'text-expression': 'format(datum.value, ".1%")',
        'domain': [0.14, 0.25]
    },
    'hours': {
        'eco-code': 'hours',
        'label': 'Average weekly hours',
        'title': 'Hours worked',
        'subtitle': 'Total weekly hours worked, SA | Source: ONS via ECO API',
        'units': 'd',
        'units-label': '.1s',
        'label-expression': 'format(datum.value / 1000, ".1f") + "bn"',
        'text-expression': 'format(datum.value / 1000, ".2f") + "bn"',
        'domain': [700, 1200]
    },
    'wage_r': {
        'eco-code': 'wage_r',
        'label': 'Real wages',
        'title': 'Wages',
        'subtitle': 'Average weekly earnings, 2015 prices, SA | Source: ONS via ECO API',
        'units': 'd',
        'units-label': '.1',
        'label-expression': '"£" + format(datum.value, ".0f")',
        'text-expression': '"£" + format(datum.value, ".0f")',
        'domain': [400, 600]
    }
}

In [200]:
# Get today's date in the format 'YYYY-MM-DD'
today = pd.to_datetime('today').strftime('%Y-%m-%d')


for key, value in lfs.items():
    api_url = f"https://api.economicsobservatory.com/gbr/{value['eco-code']}?vega"

    title=alt.TitleParams(
        value['title'],
        subtitle = value['subtitle'],
        subtitleFontSize=12,
        anchor='start',
        frame='group',
        offset=7
    )

    base = alt.Chart(api_url, title=title).encode(x='date:T').transform_calculate(
        # Calculate year from date
        year='year(datum.date)'
    ).transform_filter(
        # Filter to last 5 years
        'datum.year > 2014'
    )

    if 'domain' in value.keys():
        line = base.transform_calculate(
            value='datum.value / 100' if '%' in value['units'] else 'datum.value'
        ).mark_line().encode(
            alt.Y('value:Q').axis(labelExpr=value['label-expression'], tickCount=6).scale(domain=value['domain'])
        )
    else:
        line = base.transform_calculate(
            value='datum.value / 100' if '%' in value['units'] else 'datum.value'
        ).mark_line().encode(
            alt.Y('value:Q').axis(labelExpr=value['label-expression'], tickCount=6)
        )

    point = line.mark_point(size=40).encode(
        alt.X('date:T', aggregate='max'),
        alt.Y('value:Q', aggregate={'argmax': 'date'})
    )

    text = line.mark_text(align='left', fontSize=12).transform_calculate(
        # Calulcatee a text label by dividing by 1000, formatting to 2 decimals, and adding bn
        text_label=value['text-expression']
    ).encode(
        alt.X('date:T', aggregate='max'),
        alt.Y('value:Q', aggregate={'argmax': 'date'}),
        # alt.Text('value:Q', aggregate={'argmax': 'date'}, format=value['units-label'])
        alt.Text('text_label:N', aggregate={'argmax': 'date'})
    )

    text_month = line.mark_text(align='left', dy=-13, fontSize=13).encode(
        alt.X('date:T', aggregate='max'),
        alt.Y('value:Q', aggregate={'argmax': 'date'}),
        alt.Text('date:T', aggregate={'argmax': 'date'}, timeUnit='yearmonth', format='%b %Y')
    )

    # .scale(
    #         domain=value['bounds'] if 'bounds' in value else 
    #     )


    chart = (line + point + text + text_month).properties(width=370)

    chart.display()

    styles.save(chart, today, f"C_{value['eco-code']}", width=380, height=300)

alt.LayerChart(...)

alt.LayerChart(...)

alt.LayerChart(...)

alt.LayerChart(...)

alt.LayerChart(...)

alt.LayerChart(...)

alt.LayerChart(...)

alt.LayerChart(...)

Custom chart

In [52]:
from altair_wrapper.altair_eco import CustomChart
import pandas as pd

# ECO API has 12-month inflation, we want 1-month updates
url = 'https://api.economicsobservatory.com/gbr/wage?vega'

# load data from ECO API in pandas dataframe
df = pd.read_json(url)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 291 entries, 0 to 290
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    291 non-null    datetime64[ns]
 1   value   291 non-null    int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 4.7 KB


In [46]:
# Example of using the class with a theme
custom_chart = CustomChart(df, theme='dark', start_date='2015-01-01').create_line_chart(
    x='date',
    y='value',
    title="UK Inflation",
    subtitle="CPIH 12-month percent change | Source: ONS via ECO API",
    y_format='.2'
)

custom_chart.display()

alt.LayerChart(...)